In [ ]:
import cv2
from IPython.display import clear_output
import sys


sys.path.append('../')
from utils.utils import read_from_env_file

env_vars = read_from_env_file()

In [ ]:
#cap = cv2.VideoCapture('../data/tienda.mp4')
cap = cv2.VideoCapture(env_vars["URL_CAMERA_TIENDA"])

In [ ]:
sys.path.append("../yolov4/")
from tool.utils import *
from tool.torch_utils import *
from tool.darknet2pytorch import Darknet
import argparse
from PIL import Image
import cv2

"""hyper parameters"""
use_cuda = False
img_size=800
conf_thres=0.8
nms_thres=0.4

namesfile = '../yolov4/cfg/coco.names'
class_names = load_class_names(namesfile)

def detect_cv2(cfgfile, weightfile, imgfile):
    
    m = Darknet(cfgfile)
    m.load_weights(weightfile)

    if use_cuda:
        m.cuda()

    num_classes = m.num_classes
    img = imgfile
    sized = cv2.resize(img, (m.width, m.height))
    sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)
    

    for i in range(2):
        start = time.time()
        boxes = do_detect(m, sized, 0.4, 0.6, use_cuda, img_size)
        finish = time.time()

    
    return boxes[0]

def get_color(c, x, max_val):
            ratio = float(x) / max_val * 5
            i = int(math.floor(ratio))
            j = int(math.ceil(ratio))
            ratio = ratio - i
            r = (1 - ratio) * colors[i][c] + ratio * colors[j][c]
            return int(r * 255)


In [ ]:
#videopath = '../data/artifacts/videos/tienda.mp4'
#vid = cv2.VideoCapture(videopath)
%pylab inline 

cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]


#while(True):
for ii in range(10):
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        

        pilimg = Image.fromarray(frame)
        boxes = detect_cv2('../yolov4/cfg/yolov4.cfg', '../yolov4/cfg/yolov4.weights', frame)
        
        img = np.copy(frame)
        colors = np.array([[1, 0, 1], [0, 0, 1], [0, 1, 1], [0, 1, 0], [1, 1, 0], [1, 0, 0]], dtype=np.float32)
        width = img.shape[1]
        height = img.shape[0]

            
        for i in range(len(boxes)):
            box = boxes[i]
            x1 = int((box[0] - box[2] / 2.0) * width)
            y1 = int((box[1] - box[3] / 2.0) * height)
            x2 = int((box[0] + box[2] / 2.0) * width)
            y2 = int((box[1] + box[3] / 2.0) * height)
                
            if len(box) >= 7 and class_names:
                cls_conf = box[5]
                cls_id = box[6]
                classes = len(class_names)
                offset = cls_id * 123457 % classes
                red = get_color(2, offset, classes)
                green = get_color(1, offset, classes)
                blue = get_color(0, offset, classes)
                rgb = (red, green, blue)
                cv2.putText(frame, class_names[cls_id], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1.2, rgb, 1)

            cv2.rectangle(frame, (x1, y1), (x2, y2), rgb, 1)


        fig=figure(figsize=(24, 16))
        title("Video Stream")
        imshow(frame)
        show()
        clear_output(wait=True)